# PROJE: Assosciation Based Recommendation System

#########################
### İş Problemi


Türkiye’nin en büyük online hizmet platformu olan Armut, hizmet verenler ile hizmet almak isteyenleri buluşturmaktadır.

Bilgisayarın veya akıllı telefonunun üzerinden birkaç dokunuşla temizlik, tadilat, nakliyat gibi hizmetlere kolayca ulaşılmasını sağlamaktadır.

Hizmet alan kullanıcıları ve bu kullanıcıların almış oldukları servis ve kategorileri içeren veri setini kullanarak Association Rule Learning ile ürün tavsiye sistemi oluşturulmak istenmektedir.

#########################

In [44]:
#########################
# Veri Seti Hikayesi

# Veri seti müşterilerin aldıkları servislerden ve bu servislerin kategorilerinden oluşmaktadır.
# Alınan her hizmetin tarih ve saat bilgisini içermektedir.

# Değişkenler;

# UserId: Müşteri numarası
# ServiceId: Her kategoriye ait anonimleştirilmiş servislerdir. (Örnek : Temizlik kategorisi altında koltuk yıkama servisi)
# Bir ServiceId farklı kategoriler altında bulanabilir ve farklı kategoriler altında farklı servisleri ifade eder.
# (Örnek: CategoryId’si 7 ServiceId’si 4 olan hizmet petek temizliği iken CategoryId’si 2 ServiceId’si 4 olan hizmet mobilya montaj)
# CategoryId: Anonimleştirilmiş kategorilerdir. (Örnek : Temizlik, nakliyat, tadilat kategorisi)
# CreateDate: Hizmetin satın alındığı tarih

In [45]:
# To do list;

# GÖREV 1: Veriyi Hazırlama

# GÖREV 2: Birliktelik Kurallarının üretilmesi

In [46]:
# Kütüphanelerimizi import edelim;
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

# pandas ayarlamalarını yapalım;
pd.set_option('display.max_columns', None)

### Görev 1:

In [47]:
# Verimizi okutalım;
df = pd.read_csv("datasets/armut_data.csv")

# Verimize hızlı bir bakış atalım;
df.head()

,UserId,ServiceId,CategoryId,CreateDate
0,25446,4,5,2017-08-06 16:11:00
1,22948,48,5,2017-08-06 16:12:00
2,10618,0,8,2017-08-06 16:13:00
3,7256,9,4,2017-08-06 16:14:00
4,25446,48,5,2017-08-06 16:16:00


In [48]:
# Verinin shape'sin bakalım;

df.shape

(162523, 4)

In [49]:
# ServisID her bir CategoryID özelinde farklı bir hizmeti temsil etmektedir.

# ServiceID ve CategoryID'yi "_" ile birleştirerek hizmetleri temsil edecek yeni bir değişken oluşturalım;
df["Hizmet"] = [str(row[1]) + "_" + str(row[2]) for row in df.values]

# Verimize hızlı bir bakış atalım;
df.head()

,UserId,ServiceId,CategoryId,CreateDate,Hizmet
0,25446,4,5,2017-08-06 16:11:00,4_5
1,22948,48,5,2017-08-06 16:12:00,48_5
2,10618,0,8,2017-08-06 16:13:00,0_8
3,7256,9,4,2017-08-06 16:14:00,9_4
4,25446,48,5,2017-08-06 16:16:00,48_5


In [50]:
# Veri seti hizmetlerin alındığı tarih ve saatten oluşmaktadır, herhangi bir sepet tanımı (fatura vb. ) bulunmamaktadır.

# Association Rule Learning uygulayabilmek için bir sepet (fatura vb.) tanımı oluşturulması gerekmektedir.
# Burada sepet tanımı her bir müşterinin aylık aldığı hizmetlerdir. 
# Örneğin; 7256 id'li müşteri 2017'in 8.ayında aldığı 9_4, 46_4 hizmetleri bir sepeti,
# 2017’in 10.ayında aldığı  9_4, 38_4  hizmetleri başka bir sepeti ifade etmektedir. 

# Sepetlerin unique bir ID ile tanımlanması gerekmektedir.
# Bunun için öncelikle sadece yıl ve ay içeren yeni bir date değişkeni oluşturacağız. 
# UserID ve yeni oluşturduğunuz date değişkenini "_" ile birleştirirek ID adında yeni bir değişkene atayacağız.

# Değişkenlerimize bakalım;
df.info()

# date değişkenimizi datetime'a çevirelim;
df["CreateDate"] = pd.to_datetime(df["CreateDate"])


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162523 entries, 0 to 162522
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   UserId      162523 non-null  int64 
 1   ServiceId   162523 non-null  int64 
 2   CategoryId  162523 non-null  int64 
 3   CreateDate  162523 non-null  object
 4   Hizmet      162523 non-null  object
dtypes: int64(3), object(2)
memory usage: 6.2+ MB


In [51]:
# Kontrol edelim;
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162523 entries, 0 to 162522
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   UserId      162523 non-null  int64         
 1   ServiceId   162523 non-null  int64         
 2   CategoryId  162523 non-null  int64         
 3   CreateDate  162523 non-null  datetime64[ns]
 4   Hizmet      162523 non-null  object        
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 6.2+ MB


In [52]:
# Sadece yıl ve ay içeren yeni bir date değişkeni oluşturalım;
df["NEW_DATE"] = df["CreateDate"].dt.strftime("%Y-%m")

# Verimize hızlı bir bakış atalım;
df.head()

,UserId,ServiceId,CategoryId,CreateDate,Hizmet,NEW_DATE
0,25446,4,5,2017-08-06 16:11:00,4_5,2017-08
1,22948,48,5,2017-08-06 16:12:00,48_5,2017-08
2,10618,0,8,2017-08-06 16:13:00,0_8,2017-08
3,7256,9,4,2017-08-06 16:14:00,9_4,2017-08
4,25446,48,5,2017-08-06 16:16:00,48_5,2017-08


In [53]:
# Unique sepet ID'leri tanımlayalım;
df["SepetID"] = [str(row[0]) + "_" + str(row[5]) for row in df.values]

# Verimize hızlı bir bakış atalım;
df.head()

,UserId,ServiceId,CategoryId,CreateDate,Hizmet,NEW_DATE,SepetID
0,25446,4,5,2017-08-06 16:11:00,4_5,2017-08,25446_2017-08
1,22948,48,5,2017-08-06 16:12:00,48_5,2017-08,22948_2017-08
2,10618,0,8,2017-08-06 16:13:00,0_8,2017-08,10618_2017-08
3,7256,9,4,2017-08-06 16:14:00,9_4,2017-08,7256_2017-08
4,25446,48,5,2017-08-06 16:16:00,48_5,2017-08,25446_2017-08


In [54]:
# Yukarıda yaptığımız işlemi test etmek adına User ID'si 7256 olan kişinin sepetlerini getirelim;

df[df["UserId"] == 7256 ]

,UserId,ServiceId,CategoryId,CreateDate,Hizmet,NEW_DATE,SepetID
3,7256,9,4,2017-08-06 16:14:00,9_4,2017-08,7256_2017-08
1268,7256,46,4,2017-08-09 16:15:00,46_4,2017-08,7256_2017-08
9540,7256,46,4,2017-08-29 03:53:00,46_4,2017-08,7256_2017-08
24679,7256,9,4,2017-10-01 04:59:00,9_4,2017-10,7256_2017-10
24680,7256,38,4,2017-10-01 05:01:00,38_4,2017-10,7256_2017-10
28698,7256,9,4,2017-10-11 08:06:00,9_4,2017-10,7256_2017-10
65325,7256,15,1,2017-12-31 04:17:00,15_1,2017-12,7256_2017-12
67093,7256,2,0,2018-01-03 22:06:00,2_0,2018-01,7256_2018-01
70623,7256,38,4,2018-01-11 13:07:00,38_4,2018-01,7256_2018-01
160299,7256,18,4,2018-07-25 00:51:00,18_4,2018-07,7256_2018-07


### GÖREV 2:

In [55]:
# Aşağıdaki gibi örnek bir matriks oluşturmak istiyoruz;

# Hizmet         0_8  10_9  11_11  12_7  13_11  14_7  15_1  16_8  17_5  18_4..
# SepetID
# 0_2017-08        0     0      1     0      0     0     0     0     0     0..
# 0_2017-09        1     0      0     0      0     0     0     0     0     0..
# 0_2018-01        0     1      0     0      0     0     0     0     0     0..
# 0_2018-04        0     0      0     0      0     1     0     0     0     0..
# 10000_2017-08    0     0      0     0      0     0     0     0     0     0..

# Yani bir sepette olan ürünler 1 ile olmayan ürünler 0 ile gösteriliyor.

    Yukarıdaki matriks'i hazırlarken sütunlara baktığımızda olası bütün hizmetleri sütunlarda görüyoruz.
    Satıda ise hizmet sepetleri bulunmaktadır.

    Bir diğer konu ise sütunlara gelen hizmetlerin binary sisteme (1-0) geçirilmesi gerekecek.

In [56]:
# Verimizi sepet ID'ye göre gruplayalım;

df.groupby(["SepetID", "Hizmet"]).agg({"Hizmet": "count"})

Hizmet
SepetID    Hizmet        
0_2017-08  46_4         1
           48_5         1
0_2017-09  48_5         1
           4_5          1
0_2018-01  30_2         1
...                   ...
99_2018-01 20_5         1
99_2018-02 0_8          1
9_2018-03  25_0         1
9_2018-04  19_6         1
           25_0         1

[121348 rows x 1 columns]

In [57]:
# Daha sonrasında unstack ile pivot table haline getirelim;

df.groupby(["SepetID", "Hizmet"]).agg({"Hizmet": "count"}).unstack()

Hizmet                                                          \
Hizmet           0_8 10_9 11_11 12_7 13_11 14_7 15_1 16_8 17_5 18_4 19_6 1_4   
SepetID                                                                        
0_2017-08        NaN  NaN   NaN  NaN   NaN  NaN  NaN  NaN  NaN  NaN  NaN NaN   
0_2017-09        NaN  NaN   NaN  NaN   NaN  NaN  NaN  NaN  NaN  NaN  NaN NaN   
0_2018-01        NaN  NaN   NaN  NaN   NaN  NaN  NaN  NaN  NaN  NaN  NaN NaN   
0_2018-04        NaN  NaN   NaN  NaN   NaN  1.0  NaN  NaN  NaN  NaN  NaN NaN   
10000_2017-08    NaN  NaN   NaN  NaN   NaN  NaN  NaN  NaN  NaN  NaN  NaN NaN   
...              ...  ...   ...  ...   ...  ...  ...  ...  ...  ...  ...  ..   
99_2017-12       3.0  NaN   NaN  NaN   NaN  NaN  NaN  NaN  NaN  NaN  NaN NaN   
99_2018-01       1.0  NaN   NaN  NaN   NaN  NaN  NaN  NaN  NaN  NaN  NaN NaN   
99_2018-02       1.0  NaN   NaN  NaN   NaN  NaN  NaN  NaN  NaN  NaN  NaN NaN   
9_2018-03        NaN  NaN   NaN  NaN   NaN  NaN  NaN  NaN  NaN  NaN  NaN NaN   
9_2018-04        NaN  NaN   NaN  NaN   NaN  NaN  NaN  NaN  NaN  NaN  1.0 NaN   

                                                                            \
Hizmet        20_5 21_5 22_0 23_10 24_10 25_0 26_7 27_7 28_4 29_0 2_0 30_2   
SepetID                                                                      
0_2017-08      NaN  NaN  NaN   NaN   NaN  NaN  NaN  NaN  NaN  NaN NaN  NaN   
0_2017-09      NaN  NaN  NaN   NaN   NaN  NaN  NaN  NaN  NaN  NaN NaN  NaN   
0_2018-01      NaN  NaN  NaN   NaN   NaN  NaN  NaN  NaN  NaN  NaN NaN  1.0   
0_2018-04      NaN  NaN  NaN   NaN   NaN  NaN  NaN  NaN  NaN  NaN NaN  1.0   
10000_2017-08  NaN  NaN  NaN   NaN   NaN  NaN  NaN  NaN  NaN  NaN NaN  NaN   
...            ...  ...  ...   ...   ...  ...  ...  ...  ...  ...  ..  ...   
99_2017-12     1.0  NaN  NaN   NaN   NaN  NaN  NaN  NaN  NaN  NaN NaN  NaN   
99_2018-01     1.0  NaN  NaN   NaN   NaN  NaN  NaN  NaN  NaN  NaN NaN  NaN   
99_2018-02     NaN  NaN  NaN   NaN   NaN  NaN  NaN  NaN  NaN  NaN NaN  NaN   
9_2018-03      NaN  NaN  NaN   NaN   NaN  1.0  NaN  NaN  NaN  NaN NaN  NaN   
9_2018-04      NaN  NaN  NaN   NaN   NaN  1.0  NaN  NaN  NaN  NaN NaN  NaN   

                                                                            \
Hizmet        31_6 32_4 33_4 34_6 35_11 36_1 37_0 38_4 39_10 3_5 40_8 41_3   
SepetID                                                                      
0_2017-08      NaN  NaN  NaN  NaN   NaN  NaN  NaN  NaN   NaN NaN  NaN  NaN   
0_2017-09      NaN  NaN  NaN  NaN   NaN  NaN  NaN  NaN   NaN NaN  NaN  NaN   
0_2018-01      NaN  NaN  NaN  NaN   NaN  NaN  NaN  NaN   NaN NaN  NaN  NaN   
0_2018-04      NaN  NaN  NaN  NaN   NaN  NaN  NaN  NaN   NaN NaN  NaN  NaN   
10000_2017-08  NaN  NaN  NaN  NaN   NaN  NaN  NaN  NaN   NaN NaN  NaN  NaN   
...            ...  ...  ...  ...   ...  ...  ...  ...   ...  ..  ...  ...   
99_2017-12     NaN  NaN  NaN  NaN   NaN  NaN  NaN  NaN   NaN NaN  NaN  NaN   
99_2018-01     NaN  NaN  NaN  NaN   NaN  NaN  NaN  NaN   NaN NaN  NaN  NaN   
99_2018-02     NaN  NaN  NaN  NaN   NaN  NaN  NaN  NaN   NaN NaN  NaN  NaN   
9_2018-03      NaN  NaN  NaN  NaN   NaN  NaN  NaN  NaN   NaN NaN  NaN  NaN   
9_2018-04      NaN  NaN  NaN  NaN   NaN  NaN  NaN  NaN   NaN NaN  NaN  NaN   

                                                                              \
Hizmet        42_1 43_2 44_0 45_6 46_4 47_7 48_5 49_1  4_5 5_11 6_7  7_3 8_5   
SepetID                                                                        
0_2017-08      NaN  NaN  NaN  NaN  1.0  NaN  1.0  NaN  NaN  NaN NaN  NaN NaN   
0_2017-09      NaN  NaN  NaN  NaN  NaN  NaN  1.0  NaN  1.0  NaN NaN  NaN NaN   
0_2018-01      NaN  1.0  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN NaN  1.0 NaN   
0_2018-04      NaN  1.0  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN NaN  NaN NaN   
10000_2017-08  NaN  NaN  NaN  NaN  1.0  NaN  NaN  NaN  NaN  NaN NaN  NaN NaN   
...            ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ..

In [58]:
# Boşlukları 0 ile doldurlım;

df.groupby(["SepetID", "Hizmet"]).agg({"Hizmet": "count"}).unstack().fillna(0)

Hizmet                                                           \
Hizmet           0_8 10_9 11_11 12_7 13_11 14_7 15_1 16_8 17_5 18_4 19_6  1_4   
SepetID                                                                         
0_2017-08        0.0  0.0   0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
0_2017-09        0.0  0.0   0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
0_2018-01        0.0  0.0   0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
0_2018-04        0.0  0.0   0.0  0.0   0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0   
10000_2017-08    0.0  0.0   0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
...              ...  ...   ...  ...   ...  ...  ...  ...  ...  ...  ...  ...   
99_2017-12       3.0  0.0   0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
99_2018-01       1.0  0.0   0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
99_2018-02       1.0  0.0   0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
9_2018-03        0.0  0.0   0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
9_2018-04        0.0  0.0   0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0   

                                                                             \
Hizmet        20_5 21_5 22_0 23_10 24_10 25_0 26_7 27_7 28_4 29_0  2_0 30_2   
SepetID                                                                       
0_2017-08      0.0  0.0  0.0   0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
0_2017-09      0.0  0.0  0.0   0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
0_2018-01      0.0  0.0  0.0   0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0   
0_2018-04      0.0  0.0  0.0   0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0   
10000_2017-08  0.0  0.0  0.0   0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
...            ...  ...  ...   ...   ...  ...  ...  ...  ...  ...  ...  ...   
99_2017-12     1.0  0.0  0.0   0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
99_2018-01     1.0  0.0  0.0   0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
99_2018-02     0.0  0.0  0.0   0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
9_2018-03      0.0  0.0  0.0   0.0   0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0   
9_2018-04      0.0  0.0  0.0   0.0   0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0   

                                                                             \
Hizmet        31_6 32_4 33_4 34_6 35_11 36_1 37_0 38_4 39_10  3_5 40_8 41_3   
SepetID                                                                       
0_2017-08      0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0   
0_2017-09      0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0   
0_2018-01      0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0   
0_2018-04      0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0   
10000_2017-08  0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0   
...            ...  ...  ...  ...   ...  ...  ...  ...   ...  ...  ...  ...   
99_2017-12     0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0   
99_2018-01     0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0   
99_2018-02     0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0   
9_2018-03      0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0   
9_2018-04      0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0   

                                                                           \
Hizmet        42_1 43_2 44_0 45_6 46_4 47_7 48_5 49_1  4_5 5_11  6_7  7_3   
SepetID                                                                     
0_2017-08      0.0  0.0  0.0  0.0  1.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0   
0_2017-09      0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  1.0  0.0  0.0  0.0   
0_2018-01      0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0   
0_2018-04      0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
10000_2017-08  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
...            ...  ...  ...  ...  ...  ...  ... 

In [59]:
# Bu veri setindeki değerleri binary formata getirelim;
# Şu anda binary formatta gibi gözüküyor olabilir, biz işimizi şansa bırakmayalım, 
# ayrıca birisi bir ayda aynı hizmeti iki kere almış olabileceğini de unutmayalım;

df.groupby(['SepetID', 'Hizmet'])['Hizmet'].count().unstack().fillna(0).applymap(lambda x: 1 if x > 0 else 0)

Hizmet,0_8,10_9,11_11,12_7,13_11,14_7,15_1,16_8,17_5,18_4,19_6,1_4,20_5,21_5,22_0,23_10,24_10,25_0,26_7,27_7,28_4,29_0,2_0,30_2,31_6,32_4,33_4,34_6,35_11,36_1,37_0,38_4,39_10,3_5,40_8,41_3,42_1,43_2,44_0,45_6,46_4,47_7,48_5,49_1,4_5,5_11,6_7,7_3,8_5,9_4
SepetID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0_2017-08,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0
0_2017-09,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
0_2018-01,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
0_2018-04,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
10000_2017-08,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99_2017-12,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
99_2018-01,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
99_2018-02,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [60]:
# Şimdi bu işlemi bir df'ye atayalım;
invoice_product_df = df.groupby(['SepetID', 'Hizmet'])['Hizmet'].count().unstack().fillna(0).   \
    applymap(lambda x: 1 if x > 0 else 0)

# Veri setine hızlı bir bakış atalım;
invoice_product_df.head()

Hizmet,0_8,10_9,11_11,12_7,13_11,14_7,15_1,16_8,17_5,18_4,19_6,1_4,20_5,21_5,22_0,23_10,24_10,25_0,26_7,27_7,28_4,29_0,2_0,30_2,31_6,32_4,33_4,34_6,35_11,36_1,37_0,38_4,39_10,3_5,40_8,41_3,42_1,43_2,44_0,45_6,46_4,47_7,48_5,49_1,4_5,5_11,6_7,7_3,8_5,9_4
SepetID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0_2017-08,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0
0_2017-09,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
0_2018-01,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
0_2018-04,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
10000_2017-08,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [61]:
# Birliktelik kurallarını oluşturalım;
frequent_itemsets = apriori(invoice_product_df, min_support=0.01, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="support", min_threshold=0.01)

rules.head()

/Users/emintaskent/opt/anaconda3/lib/python3.9/site-packages/mlxtend/frequent_patterns/fpcommon.py:111: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(2_0),(13_11),0.130286,0.056627,0.012819,0.098394,1.737574,0.005442,1.046325
1,(13_11),(2_0),0.056627,0.130286,0.012819,0.226382,1.737574,0.005442,1.124216
2,(2_0),(15_1),0.130286,0.120963,0.033951,0.260588,2.154278,0.018191,1.188833
3,(15_1),(2_0),0.120963,0.130286,0.033951,0.280673,2.154278,0.018191,1.209066
4,(33_4),(15_1),0.027310,0.120963,0.011233,0.411311,3.400299,0.007929,1.493211


    Bu tablonun nasıl okunacağını bir hatırlayalım;

* antecedents : birinci üründür.

* consequents : ikinci üründür (veya ürünlerdir).

* antecedent support : ilk ürünün tek başına gözlenme olasılığıdır. (Yani support değeridir.)

* consequent support : ikinci ürün veya ürünlerin tek başına gözlenme olasılığıdır.

* support : iki ürünün birlikte gözükme olasılığıdır.

* confidence : x ürünü alındığında y'nin alınması olasılığıdır.

* lift : x ürünü satın alındığında y ürünün alınma olasığı "lift" kat artmasıdır.

* leverage : lift değerine çok benzer. Lakin support'u yüksek olan ürün gruplarına öncelik veren bir değerdir. Dolayısıyla yanlıdır. (Lift bizim için daha önemlidir.)

* conviction : Bu değere odaklanmayacağız.

        Support değerinin Lift değerine oranının birden büyük olması Lift istatistiğinin bir bağımlılık ölçüsü olduğunu (birbirlerini etkilediğini) gösterir


In [62]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(2_0),(13_11),0.130286,0.056627,0.012819,0.098394,1.737574,0.005442,1.046325
1,(13_11),(2_0),0.056627,0.130286,0.012819,0.226382,1.737574,0.005442,1.124216
2,(2_0),(15_1),0.130286,0.120963,0.033951,0.260588,2.154278,0.018191,1.188833
3,(15_1),(2_0),0.120963,0.130286,0.033951,0.280673,2.154278,0.018191,1.209066
4,(33_4),(15_1),0.027310,0.120963,0.011233,0.411311,3.400299,0.007929,1.493211
5,(15_1),(33_4),0.120963,0.027310,0.011233,0.092861,3.400299,0.007929,1.072262
6,(38_4),(15_1),0.066568,0.120963,0.011177,0.167897,1.388001,0.003124,1.056404
7,(15_1),(38_4),0.120963,0.066568,0.011177,0.092397,1.388001,0.003124,1.028458
8,(15_1),(49_1),0.120963,0.067762,0.010011,0.082763,1.221375,0.001815,1.016354
9,(49_1),(15_1),0.067762,0.120963,0.010011,0.147741,1.221375,0.001815,1.031420


In [63]:
sr = rules.sort_values("lift", ascending=False)
sr

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
10,(25_0),(22_0),0.042895,0.047515,0.011120,0.259247,5.456141,0.009082,1.285834
11,(22_0),(25_0),0.047515,0.042895,0.011120,0.234043,5.456141,0.009082,1.249553
19,(38_4),(9_4),0.066568,0.041393,0.010067,0.151234,3.653623,0.007312,1.129413
18,(9_4),(38_4),0.041393,0.066568,0.010067,0.243216,3.653623,0.007312,1.233418
5,(15_1),(33_4),0.120963,0.027310,0.011233,0.092861,3.400299,0.007929,1.072262
4,(33_4),(15_1),0.027310,0.120963,0.011233,0.411311,3.400299,0.007929,1.493211
13,(22_0),(2_0),0.047515,0.130286,0.016568,0.348700,2.676409,0.010378,1.335350
12,(2_0),(22_0),0.130286,0.047515,0.016568,0.127169,2.676409,0.010378,1.091260
15,(2_0),(25_0),0.130286,0.042895,0.013437,0.103136,2.404371,0.007849,1.067168
14,(25_0),(2_0),0.042895,0.130286,0.013437,0.313257,2.404371,0.007849,1.266432


In [64]:
# arl_recommender adında bir fonksiyon tanımlayalım ve 
# bu fonksiyon belirli bir hizmeti alan kullanıcıya başka hangi hizmetleri önerebileceğimizi göstersin;

def arl_recommender(rules_df, product_id, rec_count=1):
    sorted_rules = rules_df.sort_values("lift", ascending=False)
    # Birliktelik kuralları lifte göre büyükten kücüğe sıralar. (en uyumlu ilk ürünü yakalayabilmek için)
    # confidence veya suport'a göre de sıralanabilir insiyatife bağlıdır.
    recommendation_list = [] # tavsiye edilecek ürünler için bos bir liste oluşturduk.
    # antecedents: X
    # items denildigi için frozenset olarak getirir. index ve hizmeti birleştirir.
    # frozenset listeyi dondurur ve değiştirilemez yapar, tuple gibi yani.
    # i: index
    # product: X yani öneri isteyen hizmet
    for i, product in sorted_rules["antecedents"].items():
        for j in list(product): # hizmetlerde(product) gez:
            if j == product_id:# eger tavsiye istenen ürün yakalanırsa:
                recommendation_list.append(list(sorted_rules.iloc[i]["consequents"]))
                # index bilgisini i ile tutuyorduk bu index bilgisindeki consequents(Y) değerini recommendation_list'e ekle.

    # tavsiye listesinde tekrarlamayı önlemek için:
    # mesela 2'li 3'lü kombinasyonlarda aynı ürün tekrar düşmüş olabilir listeye gibi;
    # sözlük yapısının unique özelliginden yararlanıyoruz.
    recommendation_list = list({item for item_list in recommendation_list for item in item_list})
    return recommendation_list[:rec_count] # :rec_count istenen sayıya kadar tavsiye ürün getir.


In [65]:
# Fonskiyonumuzu deneyelim;

arl_recommender(rules,"2_0", 4)

['38_4', '15_1', '2_0', '9_4']